Import Libraries

In [176]:
import pandas as pd
import numpy as np
import os
import yfinance as yf


Currency Codes

In [177]:
country_currency_codes_table = {
    'Burundi': 'BIF',
    'Benin': 'XOF',
    'Burkina Faso': 'XOF',
    'Central African Republic': 'XAF',
    "Côte d'Ivoire": 'XOF',
    'Cameroon': 'XAF',
    'Democratic Republic of the Congo': 'CDF',
    'Republic of Congo': 'XAF',
    'Comoros': 'KMF',
    'Eritrea': 'ERN',
    'Ethiopia': 'ETB',
    'Ghana': 'GHS',
    'Guinea': 'GNF',
    'The Gambia': 'GMD',
    'Guinea-Bissau': 'XOF',
    'Liberia': 'LRD',
    'Madagascar': 'MGA',
    'Mali': 'XOF',
    'Mozambique': 'MZN',
    'Mauritania': 'MRU',
    'Malawi': 'MWK',
    'Niger': 'XOF',
    'Rwanda': 'RWF',
    'Sudan': 'SDG',
    'Senegal': 'XOF',
    'Sierra Leone': 'SLL',
    'Somalia': 'SOS',
    'Sao Tomee and Principe': 'STN',
    'Chad': 'XAF',
    'Togo': 'XOF',
    'Tanzania': 'TZS',
    'Uganda': 'UGX',
    'Zambia': 'ZMW'
}


Read Data

In [178]:
#read in stata dataset as dataframe "country_code_df"
country_code_df = pd.read_stata("codesffactors_Mar23_2019.dta") 
country_code_df.replace('--', np.nan, inplace=True) #replace all entries that are '--' with NaN data value

country_code_df.head()

,countryname,wbctry,wbcode,wbregion,wbigroup2017,wblcat2017,hipc,emu,cemac,waemu,imfcode,imfweocode,imfweoiso,imfweoctry,region
0,Aruba,Aruba,ABW,Latin America & Caribbean,High income,NaN,0,0,0,0,NaN,314.0,ABW,Aruba,LAC
1,Afghanistan,Afghanistan,AFG,South Asia,Low income,IDA,1,0,0,0,512.0,512.0,AFG,Afghanistan,SAS
2,Angola,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,0,0,0,0,614.0,614.0,AGO,Angola,SSA
3,Anguilla,NaN,AIA,Latin America & Caribbean,NaN,NaN,0,0,0,0,312.0,NaN,NaN,NaN,LAC
4,Albania,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,0,0,0,0,914.0,914.0,ALB,Albania,ECA


In [179]:
region_classification_ls = country_code_df['wbregion'].unique() #get a list of region classifications
income_classification_ls = country_code_df['wbigroup2017'].unique() #get a list of income classifications
income_classification_ls = [x for x in income_classification_ls if not pd.isna(x)] # remove '--' from income_classification_ls

print(f"The region classifications are: {', '.join(country for country in region_classification_ls)}")
print(f"The income classifications are: {', '.join(country for country in income_classification_ls)}")

The region classifications are: Latin America & Caribbean, South Asia, Sub-Saharan Africa, Europe & Central Asia, Middle East & North Africa, East Asia & Pacific, North America
The income classifications are: High income, Low income, Lower middle income, Upper middle income


In [180]:
ssa_hipc_countries = country_code_df[(country_code_df['hipc'] == 1) & (country_code_df['wbregion'] == 'Sub-Saharan Africa')]
ssa_hipc_countries_ls = ssa_hipc_countries['imfweoctry'].to_list()

ssa_hipc_countries_ls

['Burundi',
 'Benin',
 'Burkina Faso',
 'Central African Republic',
 "Côte d'Ivoire",
 'Cameroon',
 'Democratic Republic of the Congo',
 'Republic of Congo',
 'Comoros',
 'Eritrea',
 'Ethiopia',
 'Ghana',
 'Guinea',
 'The Gambia',
 'Guinea-Bissau',
 'Liberia',
 'Madagascar',
 'Mali',
 'Mozambique',
 'Mauritania',
 'Malawi',
 'Niger',
 'Rwanda',
 'Sudan',
 'Senegal',
 'Sierra Leone',
 'Somalia',
 'São Tomé and Príncipe',
 'Chad',
 'Togo',
 'Tanzania',
 'Uganda',
 'Zambia']

Read in IMF's Government Finance Statistics

In [181]:
master_table = {}
folder_path = "revenue-data"

# Loop that goes through all files 'file_name' in the folder 'folder_path'
for file_name in os.listdir(folder_path):
  if file_name.endswith('.xlsx') and not file_name.startswith('~$'): #check to see if is spreadsheet .xlsx format
    raw_df = pd.read_excel(folder_path+ "/" + file_name) #read in .xlsx as dataframe
    master_table[file_name[:-5]] = raw_df #store dataframe as an entry in the master_table hash table

# master_table["Central African Republic"]
master_table["Central African Republic"]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,Detailed Revenue Breakdown,Detailed Revenue Breakdown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Country: Central African Rep.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sector: Budgetary central government,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Unit: Domestic currency,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Scale: Millions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2010,2011.000000,2012.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000
7,Revenue,159856.425083,125400.435089,201430.000000,178749.663179,92314.460187,167879.377909,166368.619605,207727.656493,239334.972783
8,Taxes,89135.596491,86631.395622,104734.000000,38344.670841,54631.151708,73625.163568,85210.311549,106583.975699,108263.477445
9,"Taxes on income, profits, & capital gains",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32961.332840


Convert values to USD

In [182]:
#REMOVE "JUNK" ROWS AND CONVERT TO CORRECT SCALE (MILLION/BILLIONS)
for key in master_table: #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
  scale = master_table[key].iloc[4,0] #get if values have scale of billions or millions
  
  # Drop rows 0 to 5, reformatting indexes and column headers
  master_table[key] = master_table[key].drop([0, 1, 2, 3, 4, 5]) 
  master_table[key] = master_table[key].reset_index(drop=True) 
  master_table[key].columns = master_table[key].iloc[0]
  master_table[key] = master_table[key][1:]
  master_table[key] = master_table[key].set_index(master_table[key].columns[0])
  master_table[key].columns = [int(float(col)) for col in master_table[key].columns]
  # master_table[key] = master_table[key].fillna(0)

  if scale == "Scale: Billions":
    master_table[key] = master_table[key]* (10 ** 9) #multiple every value in dataframe by 1e9 if scale is billion
  elif scale == "Scale: Millions":
    master_table[key] = master_table[key]* (10 ** 6) #multiple every value in dataframe by 1e9 if scale is billion


#FILTER OUT COUNTRIES WITH LESS THAN 7 YEARS OF DATA
less_than_seven_years_data_countries = [] #create list to store countries with less than 7 years of historial data
for key in master_table: #iterate through master_table
  num_columns = master_table[key].shape[1] #count the number of columns (each column represents a year)
  if num_columns < 7: #if less than 7 columns/year, add to list to delete
    less_than_seven_years_data_countries.append(key)

for country in less_than_seven_years_data_countries:
  del master_table[country] #delete countries with less than 7 years from data table


#CONVERT LOCAL CURRENCY DENOMINATED VALUES TO USD
error_countries = []
for key in master_table:
  #get annual 2010-2019 USD to domestic currency exchange rate using yfinance library
  currency = country_currency_codes_table[key] #retrieve currency symbol from 'country_currency_codes_table'
  symbol = f"{currency}=X"
  try:
    data = yf.download(symbol, start="2010-01-01", end="2020-01-01", interval="1mo") #download monthly USD-Currency dataset from 2010-2020
    yearly_data_df = data['Close'].resample('YE').mean() #get the annual average exchange rate outputted as datafram
    yearly_data_table = yearly_data_df.to_dict() # Convert the yearly_data to a hash table with the date as the key
    year_to_exchange_rate_dict = {key.year: value for key, value in yearly_data_table.items()} # Convert the keys to only the year part
    print(year_to_exchange_rate_dict)

    #convert DataFrame values to USD
    for year in master_table[key].columns: #loop through columns in dataframe
      year_int = int(year)  # Convert year to integer to match the dictionary keys
      master_table[key][year] = master_table[key][year] / year_to_exchange_rate_dict[year_int] #divide each column's values by that year's exchange rate

  except Exception as e: #if errored out
    print(f"Could not fetch rate for {key} ({currency}): {e}")
    error_countries.append(key) #add country to error list
    pass
    
for country in error_countries:
  del master_table[country] #delete countries where currency conversion failed
 
master_table["Central African Republic"]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['STN=X']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of 

{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 487.61083221435547, 2011: 460.5933329264323, 2012: 501.0466715494792, 2013: 492.91916910807294, 2014: 497.47583770751953, 2015: 594.4266713460287, 2016: 594.0358378092448, 2017: 577.82666015625, 2018: 557.75, 2019: 583.7949981689453}
{2010: 487.5991668701172, 2011: 460.59083048502606, 2012: 501.04666900634766, 2013: 492.91916910807294, 2014: 497.4808349609375, 2015: 594.4341735839844, 2016: 594.0333353678385, 2017: 578.2424977620443, 2018: 556.1933390299479, 2019: 586.7139180501302}
{2010: 1422.58332316080

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


{2010: 34.550000508626304, 2011: 28.579166730244953, 2012: 28.048333326975506, 2013: 29.664999802907307, 2014: 31.18499978383382, 2015: 39.24833265940348, 2016: 62.973334312438965, 2017: 62.673333168029785, 2018: 59.819166819254555, 2019: 61.90666675567627}
{2010: 3920.0, 2011: 4280.166666666667, 2012: 4277.166666666667, 2013: 4268.166666666667, 2014: 4302.333333333333, 2015: 4348.583333333333, 2016: 5087.916666666667, 2017: 7505.0, 2018: 7962.916666666667, 2019: 9048.333333333334}
{2010: 1206.0750122070312, 2011: 1216.6916605631511, 2012: 1396.4749857584636, 2013: 1531.5916748046875, 2014: 1530.9249877929688, 2015: 1536.9166666666667, 2016: 1605.7583312988281, 2017: 1700.5, 2018: 1756.2916666666667, 2019: 1822.7166646321614}
{2010: 487.5991668701172, 2011: 460.59083048502606, 2012: 501.04666900634766, 2013: 492.91916910807294, 2014: 497.4808349609375, 2015: 594.4341735839844, 2016: 594.0333353678385, 2017: 578.2424977620443, 2018: 556.1933390299479, 2019: 586.7139180501302}
{2010: 207

,2010,2011,2012,2014,2015,2016,2017,2018,2019
nan,,,,,,,,,
Revenue,327843925.79906,2.722599e+08,4.020184e+08,3.593096e+08,1.552980e+08,2.826094e+08,2.877143e+08,3.734810e+08,4.079245e+08
Taxes,182805063.148812,1.880875e+08,2.090304e+08,7.707768e+07,9.190446e+07,1.239411e+08,1.473609e+08,1.916312e+08,1.845252e+08
"Taxes on income, profits, & capital gains",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.617957e+07
"Taxes on income, profits, & capital gains: individuals",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.802275e+07
"Taxes on income, profits, & capital gains: corporations",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.738561e+07
"Taxes on income, profits, & capital gains: other",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.712041e+05
Taxes on payroll & workforce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00
Taxes on property,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.886592e+06
Taxes on goods & services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.945557e+07


Calculate Yearly Averages

In [183]:
data_by_year_table = {
  2010: [],
  2011: [],
  2012: [],
  2013: [],
  2014: [],
  2015: [],
  2016: [],
  2017: [],
  2018: [],
  2019: [],
} #make table that stores data by year

# master_averages_ls = [] #make master averages list as place to store averages

for key in master_table:
  years_available = master_table[key].columns
  for year in data_by_year_table:
    if year in years_available:
      data_by_year_table[year].append(master_table[key][year])

yearly_averages_ls = [] #make a list of the yearly averages
for year in data_by_year_table:
  merged_year_df = pd.concat(data_by_year_table[year], axis=1, join='outer')
  year_average_df = merged_year_df.mean(axis=1) #get the average of all the countries' averages
  yearly_averages_ls.append(year_average_df)

 
# #Get average over decade
# #loop through table (where key is the country name) (master_table[key] is the dataframe stored in the master_table indexed by key)
# for key in master_table:
#   per_category_average = master_table[key].mean(axis=1) #get the average of each category (ignoring np.NaN values) into a list
#   # per_category_average.drop('To other government units', inplace=True) #drop 'To other government units' since there's two rows named that and will error
#   master_averages_ls.append(per_category_average) #add each country average to to the master averages list



# merged_averages_df = pd.concat(master_averages_ls, axis=1, join='outer')
# average__of_averages_df = merged_averages_df.mean(axis=1) #get the average of all the countries' averages
# average__of_averages_df

for i in range(len(yearly_averages_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_ls[i])

YEAR: 2010
nan
Revenue                                                    2883443168.516404
Taxes                                                      1519390553.555896
Taxes on income, profits, & capital gains                   418873946.772721
Taxes on income, profits, & capital gains: individuals      226664302.324849
Taxes on income, profits, & capital gains: corporations     192252125.253461
Taxes on income, profits, & capital gains: other             23164608.017139
Taxes on payroll & workforce                                  3881300.146776
Taxes on property                                             4428760.240327
Taxes on goods & services                                   740236005.160065
General taxes on goods & services                           450292661.842953
Excise taxes                                                148880608.885855
Taxes on int trade & transactions                           368080714.113236
Other taxes n.e.c.                                           

Calculate Percentages

In [184]:
yearly_averages_percentage_ls = []
for year_average_df in yearly_averages_ls:
  total_taxes = year_average_df['Taxes']
  year_average_percentage_df = year_average_df / total_taxes * 100 #take all values as a fraction of total expenditure, then convert decimal to percent, then round to two decimal places
  yearly_averages_percentage_ls.append(year_average_percentage_df)

for i in range(len(yearly_averages_percentage_ls)):
  print(f"YEAR: {2010+i}")
  print(yearly_averages_percentage_ls[i])

YEAR: 2010
nan
Revenue                                                     189.7763
Taxes                                                          100.0
Taxes on income, profits, & capital gains                   27.56855
Taxes on income, profits, & capital gains: individuals     14.918107
Taxes on income, profits, & capital gains: corporations    12.653239
Taxes on income, profits, & capital gains: other            1.524599
Taxes on payroll & workforce                                0.255451
Taxes on property                                           0.291483
Taxes on goods & services                                  48.719271
General taxes on goods & services                            29.6364
Excise taxes                                                9.798706
Taxes on int trade & transactions                           24.22555
Other taxes n.e.c.                                          4.804267
Social contributions                                         0.54505
Social security con

Calculate Average Across Period

In [185]:
merged_percentages_df = pd.concat(yearly_averages_percentage_ls, axis=1, join='outer')
average_df = merged_percentages_df.mean(axis=1)
rounded_average_df = average_df.apply(lambda x: round(x, 2))

rounded_average_df

nan
Revenue                                                    147.24
Taxes                                                      100.00
Taxes on income, profits, & capital gains                   31.64
Taxes on income, profits, & capital gains: individuals      15.51
Taxes on income, profits, & capital gains: corporations     16.94
Taxes on income, profits, & capital gains: other             1.82
Taxes on payroll & workforce                                 0.45
Taxes on property                                            0.70
Taxes on goods & services                                   50.80
General taxes on goods & services                           31.82
Excise taxes                                                10.96
Taxes on int trade & transactions                           18.77
Other taxes n.e.c.                                           2.52
Social contributions                                         0.60
Social security contributions                                0.01
Other 

In [186]:
#one method filter for countries with 7+ years of data, aggregate in way that preserves identities
#unweighted - convert to share and take average (ratios period by period and then average)